In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
# 1. Preliminary EDA

# Set the file path
file_path = os.path.join(os.pardir, 'data_files', 'HomeA-meter4_2016.csv')

# Load the dataset
df = pd.read_csv(file_path)

# Drop 'gen [kW]' column
df.drop(columns=['gen [kW]'], inplace=True)

# Sum the values in each row and update 'use [kW]' column
df['use [kW]'] = df.iloc[:, 2:].sum(axis=1)

# Define the path for the cleaned data folder
cleaned_data_folder = 'cleaned_data'

# Check if the cleaned data folder exists, if not, create it
if not os.path.exists(cleaned_data_folder):
    os.makedirs(cleaned_data_folder)

# Save the updated DataFrame to a new CSV file under the cleaned_data folder
output_file_path = os.path.join(cleaned_data_folder, 'HomeA-meter4_2016_updated.csv')
df.to_csv(output_file_path, index=False)

# Create a file to store the output
output_file = 'data_consistency_report.txt'


In [3]:


with open(output_file, 'w') as f:
    # Display the first few rows of the dataset
    f.write("Original Dataset:\n")
    f.write(str(df.head()) + "\n\n")

    # Data Cleaning Steps
    # 1. Convert Date & Time column to datetime format
    df['Date & Time'] = pd.to_datetime(df['Date & Time'])

    # 2. Handle Missing Values (if any)
    # Check for missing values
    missing_values = df.isnull().sum()
    f.write("Missing Values:\n")
    f.write(str(missing_values) + "\n\n")

    # 3. Check Data Consistency
    # Check column data types
    f.write("Data Types:\n")
    f.write(str(df.dtypes) + "\n\n")

    # Range checks for numerical columns
    numerical_columns = df.select_dtypes(include=['float64']).columns
    f.write("Range Checks:\n")
    for col in numerical_columns:
        min_val = df[col].min()
        max_val = df[col].max()
        f.write(f"\nRange for {col}:\n")
        f.write(f"Min: {min_val}, Max: {max_val}\n")

    # Check unique values for categorical columns (if any)
    categorical_columns = df.select_dtypes(include=['object']).columns
    if len(categorical_columns) > 0:
        f.write("\nUnique Values for Categorical Columns:\n")
        for col in categorical_columns:
            unique_values = df[col].unique()
            f.write(f"\n{col}:\n")
            f.write(str(unique_values) + "\n")

    # Temporal consistency: Check for duplicate timestamps
    duplicate_timestamps = df['Date & Time'].duplicated().sum()
    f.write("\nDuplicate Timestamps: " + str(duplicate_timestamps) + "\n\n")

print("Data consistency report completed. Report saved to:", output_file)


Data consistency report completed. Report saved to: data_consistency_report.txt


In [4]:
# 4. Normalize Data

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Select numerical columns to normalize
numerical_columns = df.select_dtypes(include=['float64']).columns

# Apply normalization to numerical columns
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

print("Data normalization completed.")

Data normalization completed.


In [5]:
# 5. Outlier Detection

# Define a function to detect outliers using the interquartile range (IQR) method
def detect_outliers(df, threshold=1.5):
    outliers = []
    for col in df.columns:
        if df[col].dtype in ['float64', 'int64']:
            Q1 = np.percentile(df[col], 25)
            Q3 = np.percentile(df[col], 75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            # Print the calculated bounds for debugging
            print(f"Column: {col}, Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
            # Identify outliers and append their indices
            outliers.extend(df[(df[col] < lower_bound) | (df[col] > upper_bound)].index)
    return list(set(outliers))

# Detect outliers in the dataset
outlier_indices = detect_outliers(df)

# Remove outliers from the dataset
cleaned_df = df.drop(outlier_indices)


Column: use [kW], Lower Bound: -0.04228294301156803, Upper Bound: 0.13208863069662286
Column: KitchenDenLights [kW], Lower Bound: -0.3128392327915092, Upper Bound: 0.5678947104704457
Column: MasterBedBathLights [kW], Lower Bound: -0.0944027800629083, Upper Bound: 0.20594402855068955
Column: MasterOutlets [kW], Lower Bound: 0.0048460198730595535, Upper Bound: 0.006860590609229876
Column: DenOutdoorLights [kW], Lower Bound: 0.021921660038775007, Upper Bound: 0.03311568968067742
Column: DenOutlets [kW], Lower Bound: -0.00817198471581757, Upper Bound: 0.013792243486629092
Column: RearBasementLights [kW], Lower Bound: 0.034369575129533675, Upper Bound: 0.04127809326424872
Column: KitchenOutletsEast [kW], Lower Bound: 0.0019562340848806358, Upper Bound: 0.0023983189655172417
Column: KitchenOutletsSouth [kW], Lower Bound: 0.001181095369471238, Upper Bound: 0.001394225360954769
Column: DishwasherDisposalSinkLight [kW], Lower Bound: -0.024102477724538038, Upper Bound: 0.04443823664831721
Column

In [6]:
# 6. Feature Engineering (if necessary)
# Extract additional temporal features: day of the week and hour of the day
cleaned_df['DayOfWeek'] = cleaned_df['Date & Time'].dt.dayofweek  # Monday=0, Sunday=6
cleaned_df['HourOfDay'] = cleaned_df['Date & Time'].dt.hour


In [7]:
# 7. Data Splitting

# Define features (X) and target variable (y)
X = df.drop(columns=['Date & Time'])  # Features (exclude 'Date & Time' column)
y = df['use [kW]']  # Target variable

# Split the dataset into training (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training set into training (80%) and validation (20%) sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Print the shapes of the resulting datasets
print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape, y_test.shape)

Training set: (321755, 13) (321755,)
Validation set: (80439, 13) (80439,)
Test set: (100549, 13) (100549,)


In [8]:
# Set the cleaned file path
cleaned_file_path = os.path.join('cleaned_data', 'HomeA-meter4_2016_updated.csv')

# Save the cleaned dataset under the cleaned_data directory
df.to_csv(cleaned_file_path, index=False)

print("\nCleaning process completed. Cleaned dataset saved to:", cleaned_file_path)


Cleaning process completed. Cleaned dataset saved to: cleaned_data/HomeA-meter4_2016_updated.csv
